In [3]:
import os
import import_ipynb
import pandas as pd
import itertools
import talib
import math
import numpy as np

'''to be modified and shift into property files'''
currentpath=os.getcwd()
dbpath=os.path.join(currentpath,"..",'database')
stockcsv=os.path.join(dbpath,'stockpath.csv')
featurescsv=os.path.join(dbpath,'features.csv')
featuresdata=os.path.join(dbpath,'featuresdata.csv')



#to import Load_Csv file from another directory. this style is for jupyter notebook
                
%cd ..
from utility import Load_Csv as lcsv
%cd stockprediction



C:\Users\shaskhan\AnacondaProjects\am\finProduct
C:\Users\shaskhan\AnacondaProjects\am\finProduct\stockprediction


In [4]:
class ta(lcsv.Load_csv):
    '''this class contains functions fto predict label with the help of technical indicators'''

    def loadcsv(self):
        '''load 'Date','Close', 'Volume' data from databse and return dataframe
        '''
        
        self.dataset=self.LoadData(stockcsv).loc[:, ['Date','Close', 'Volume']]
        self.dataset['Date'] = pd.to_datetime(self.dataset['Date'])
        return self.dataset
    

    def loadfeaturesdata(self,x):
        '''load features detailed data from databse'''
        self.featuresdata=self.LoadfeaData(featuresdata)
        self.featuresdata.index=self.featuresdata[0]
        #self.featuresdata=self.featuresdata.loc[x]
        return list(self.featuresdata.loc[x,1:])
    
    
    def ti_Combinations(self):
        '''takes input as list of list and  gives output as comninations '''
        
        print("ti_Combinations")

        self.paneldict={}
        def comb_r(row):
            #print(self.dataset)
            comb_dataset=self.dataset.copy(deep=True)
            for i in row:
                comb_dataset['MA'+str(i)]=self.tdf['MA'+str(i)]
            self.paneldict[row]=comb_dataset.copy(deep=True)
            #print('a',self.paneldict)
                        

            
                    
        try:
            a=self.tidict['MA']
            #[[10],[50],[60-64]]
            comb_df=pd.Series(list(itertools.product(*a)))##get combinatons 
            comb_df.apply(comb_r)
            panel_dataset=pd.Panel(self.paneldict)
            print(panel_dataset)
            
            
        except Exception as e:
            print('e2',e)
            

        
    
        
        
    def loadfeatures(self):
        '''load feature label data from databse
        '''
        self.featurestilist=[]
        self.misc=[]
        self.label=[]
        self.tilist=dir(talib)
        self.featuredict={}
        self.tidict={}
        self.features=self.LoadfeaData(featurescsv)

         
        def func(value,args):
           

            if str(value).find("-")>-1:   #tocheck if range is given 
                a,b=value.split("-")
                a=int(a)
                b=int(b)

                self.featuredict[args].extend(list(range(a,b)))   #if range is given then replace it by list
                self.tidict[args].append(list(range(a,b))) 

            else:
                try:
                    var=str(args)+'-'+str(int(value))              #togenerate name like MA-1,MA-2  

                    self.featuredict[args].extend(self.loadfeaturesdata(var))
                    
                    self.tidict[args].append(self.loadfeaturesdata(var))
                except Exception as e:
                    print('e1',e)    
        
        
        def funr(row):
            r_len=len(row)
            ti=row[0]
            rowdf=pd.Series(row[1:])
            
                     
            if ti in self.tilist:
                self.featuredict[ti]=[]
                self.tidict[ti]=[]
                self.featurestilist.append(ti)
                
                rowdf.apply(func,args=(ti,))    #Vectorize function over each element
                
            elif ti=='label':
                self.label.append(row[1])
            else:
                self.misc.append(row)
    
                
                                                 #vectorize approach to speed up process        
        self.features.apply(funr, axis=1)
        print(self.featuredict)                  #To be removed after development
        #print("end")
        
        
        
        #return(self.featurestilist,self.label,self.misc,self.MA_comb)
        
        
        
    def get_MA(self,x):
        malist=self.featuredict['MA']
        for i in malist:
            if ('MA'+str(i)) in self.tdf:
                continue
            else:          
                self.tdf['MA'+str(i)]=talib.SMA(self.dataset[x],i)
        return(self.tdf)
        
    def get_RSI(self,x):
        rsilist=self.featuredict['RSI']
        for i in rsilist:
            if ('RSI'+str(i)) in self.tdf:
                continue
            else:
                self.dataset['RSI'+str(i)]=talib.RSI(self.dataset[x],i)
        return(self.dataset)
        
        
        
    def get_technical_indi(self):
        
        self.tdf=self.LoadData(stockcsv).loc[:, ['Close']]
        for i in self.featurestilist:
            if i=='MA':
                self.get_MA('Close')
            if i=='RSI':
                self.get_RSI('Close')
                
                
    def get_return(self):
        self.dataset['Daily_Return'] = self.dataset['Close'].pct_change()
        
        
    def get_label(self):
        print("start")
        print(self.label)
        if self.label[0]=='return':
            self.get_return()
        print(self.dataset)
        #print(self.tdf)
        #print(self.tidict)
       
        
 

            

            
    
    
    

    
  
        
        
        

In [6]:
b=ta()
b.loadcsv()
b.loadfeatures()
b.get_technical_indi()
b.get_label()
b.ti_Combinations()


e1 cannot convert float NaN to integer
{'MA': [10, 50, 60, 61, 62, 63, 64], 'RSI': [10, 20]}
start
['return']
         Date     Close       Volume      RSI10      RSI20  Daily_Return
0  2018-01-01  10435.55  134532090.0        NaN        NaN           NaN
1  2018-01-02  10442.20  158092430.0        NaN        NaN      0.000637
2  2018-01-03  10443.20  172516859.0        NaN        NaN      0.000096
3  2018-01-04  10504.80  180257392.0        NaN        NaN      0.005899
4  2018-01-05  10558.85  186469717.0        NaN        NaN      0.005145
5  2018-01-08  10623.60  174181231.0        NaN        NaN      0.006132
6  2018-01-09  10637.00  211291563.0        NaN        NaN      0.001261
7  2018-01-10  10632.20  181900014.0        NaN        NaN     -0.000451
8  2018-01-11  10651.20  158630055.0        NaN        NaN      0.001787
9  2018-01-12  10681.25  180592153.0        NaN        NaN      0.002821
10 2018-01-15  10741.55  181262074.0  98.479087        NaN      0.005645
11 2018-01-16 

C:\Users\shaskhan\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  
